# Gradient Interpretation

In this notebook we try to learn which part of the text contributes the most to the "GPT-like" decision by looking into the gradient of every input token back propagated from the loss function given "true label" is "positive".

If the gradient $|\nabla tok|_2^2$ is **large**, then we consider this word contributes **more** to the GPT-ness since it doesn't need to change much to make the decision to be positive (GPT).

If the gradient $|\nabla tok|_2^2$ is **small**, then we consider this word contributes **less** to the GPT-ness since it need to change a lot to make the decision of classifier to be positive (GPT).

<span style="background-color:black; color: white;">Text </span> like this means this word is more GPT-like

<span style="background-color:rgb(230, 230, 230); ">Text </span> like this means this word is less GPT-like

In [1]:
import torch
import json
import numpy as np

from pathlib import Path
from IPython.display import display, HTML

from Model import T5SentinelRequireGrad

## T5SentinelRequireGrad

We do a lot of sneaky things to get the gradient propagate through original embedding of T5 language model. In short, we substitute the original embedding with some hand-crafted wrapper embedding class that will manually store the intermediate result and gather gradient information from the model.

In [2]:
PROJECT_ROOT = Path.cwd().parent.parent.parent
PATH_CHECKPT = Path(PROJECT_ROOT, "result", "cache", "t5.small.0422.pt")
PATH_CONF_LUT = Path(PROJECT_ROOT, "result", "cache", "confidence_t5_original.json")
SHOW_CONFIDENCE = PATH_CONF_LUT.exists()


model = T5SentinelRequireGrad()
model.eval()
checkpoint = torch.load(PATH_CHECKPT)
model.load_state_dict(checkpoint["model"])
model = model.to("cuda")

if SHOW_CONFIDENCE:
    with open(PATH_CONF_LUT, "r") as f:
        confidence_cache = json.load(f)
        confidence_table = confidence_cache["LUT"]
        confidence_bin   = confidence_cache["bin_size"]
else:
    print("Not showing confidence score, execute `./evaluation/evaluate_confidence.ipynb` to enable this feature")

def get_confidence_value(p: float):
    return confidence_table[int(p // confidence_bin)]

Embedder Substitution Complete
T5SentinelEmbedder(
  (embedder): Embedding(32128, 512)
)
Not showing confidence score, execute `./evaluation/evaluate_confidence.ipynb` to enable this feature


In [18]:
MAX_GRADIENT=1e6
MIN_GRADIENT=1e-6

def render_word(word, grad):
    # Normalize the gradient
    grad = max(grad, MIN_GRADIENT)
    log_scale_grad = np.log2(grad)
    normalize_grad = (log_scale_grad - np.log2(MIN_GRADIENT)) / np.log2(MAX_GRADIENT)
    grad = normalize_grad
    
    if grad > 0.5: word_color = "white"
    else: word_color = "black"
    return f'<span style="background-color: rgb({255 - grad * 255}, {255 - grad * 255}, {255 - grad * 255}); color:{word_color}">{word} </span>'

def render_latex(word: str, grad):
    # Normalize the gradient
    grad = max(grad, MIN_GRADIENT)
    log_scale_grad = np.log2(grad)
    normalize_grad = (log_scale_grad - np.log2(MIN_GRADIENT)) / np.log2(MAX_GRADIENT)
    grad = 255 - int(normalize_grad * 128)

    # if grad > 150: pre_text, post_text = "\\textcolor{white!100}{", "}"
    # else: pre_text, post_text = "", ""
    pre_text, post_text = "", ""

    word = word.replace("_", "\_")
    word = word.replace("$", "\$")

    return "\\ctext[RGB]{"+f"{grad},{grad},{grad}"+ "}{" + pre_text + word + post_text + " }"
    

def evaluate(text, model, latex=False):
    prop, grad, tokens = model(text, "negative")

    token_gradient = np.linalg.norm(grad.cpu().numpy(), axis=1)
    # token_gradient = token_gradient / np.max(token_gradient)
    token_array    = [model.t5_tokenizer.decode(tok) for tok in tokens.cpu().numpy() if tok != 0]

    HTML_STRING = ""
    LATEX_STRING = ""

    for index, tok in enumerate(token_array):
        HTML_STRING += render_word(tok, token_gradient[index])
        if latex: LATEX_STRING += render_latex(tok, token_gradient[index])

    display(HTML(HTML_STRING))
    print("Predict as", "human" if prop[1] > prop[0] else "generated")
    print("With probability of", max(prop).item())
    if SHOW_CONFIDENCE: print("With confidence of", get_confidence_value(prop[0]))

    if latex:
        print("LaTeX Renderer:\n\n")
        print(LATEX_STRING)

**Unique Identifier**: [urlsf_subset00]-[309279], OpenWebText

In [19]:
TEXT = """Apple started an avalanche of activity with the introduction of the iPad. Companies shifted gears to go after this undiscovered new tablet market. In spite of the number of players in tablets, no company has discovered the magic bullet to knock the iPad off the top of the tablet heap.\nColleague Adrian Kingsley-Hughes has a thoughtful piece blaming Amazon and Google for killing the tablet market. His reasoning is that by releasing the Kindle Fire and the Nexus 7 at $199, Amazon and Google have started a \"race to the bottom\" of the tablet market that will ensure no profitability for anyone.\nAdrian's reasoning is solid, but it overlooks one thing I have said for a long time. There is no proven tablet market. Even after all the activity in the tablet space, there is still only a proven iPad market.\nThe only company with any tablet sales to speak (besides Apple) is Samsung, and the company has not been quick to claim a lot of profits. Even Samsung had to release a whole slew of tablets in every size and shape to try to steal iPad sales, seemingly to no avail.\nHistorically, when a race to the bottom (reducing pricing so low prifitability is not a given) is dictated by the market, it's more a sign of a lack of a market in general. If enough buyers aren't willing to pay enough for a product to make producers a profit, the market is just not sufficient.\nThat's what we are seeing in the tablet space. Apple has struck a balance with the iPad to create sufficient demand that buyers are willing to pay profitable prices for, but no one else has even come close. There's an iPad market, but not a conventional tablet market.\nThat doesn't mean that Google and Amazon won't make money on the Nexus 7 and Kindle Fire respectively, just not in the conventional sense. What the situation is indicating is that the real (and only) market for tablets without an Apple logo is from auxiliary sales created by having the products in buyers' hands.\nFor Amazon those sales are content and other product sales the Kindle Fire is stimulating through customer usage. The company is not specifying what those sales might be but odds are they are significant.\nThe Nexus 7 is too new to the market to make definitive claims, but odds are Google will get good mileage selling mobile ads and Android good will from the tablet. The end result will likely be a net positive for Google, even if the Nexus 7 is selling at a break-even price or a slight loss.\nThe company with a big exposure so far is Asus, the maker of the Nexus 7. Unless Google is paying Asus a lot of cash to make production of the tablet profitable, it might not be able to make them for long. Asus doesn't have any auxiliary sales from the Nexus 7, so the exposure to the race to the bottom is very real.\nDon't get me wrong, I believe there is sufficient demand for non-iPad tablets. I just haven't seen any proof that consumers are willing to pay enough for them to make companies any money.\nSee related:"""

evaluate(TEXT, model, latex=True)

Predict as human
With probability of 0.9982826709747314
LaTeX Renderer:


\ctext[RGB]{255,255,255}{Apple }\ctext[RGB]{249,249,249}{started }\ctext[RGB]{255,255,255}{an }\ctext[RGB]{255,255,255}{ }\ctext[RGB]{255,255,255}{ava }\ctext[RGB]{255,255,255}{lan }\ctext[RGB]{255,255,255}{che }\ctext[RGB]{255,255,255}{of }\ctext[RGB]{255,255,255}{activity }\ctext[RGB]{255,255,255}{with }\ctext[RGB]{255,255,255}{the }\ctext[RGB]{255,255,255}{introduction }\ctext[RGB]{255,255,255}{of }\ctext[RGB]{255,255,255}{the }\ctext[RGB]{255,255,255}{iPad }\ctext[RGB]{249,249,249}{. }\ctext[RGB]{253,253,253}{Companies }\ctext[RGB]{255,255,255}{ }\ctext[RGB]{255,255,255}{shifted }\ctext[RGB]{255,255,255}{gear }\ctext[RGB]{255,255,255}{s }\ctext[RGB]{255,255,255}{to }\ctext[RGB]{255,255,255}{go }\ctext[RGB]{255,255,255}{after }\ctext[RGB]{255,255,255}{this }\ctext[RGB]{255,255,255}{un }\ctext[RGB]{246,246,246}{discovered }\ctext[RGB]{255,255,255}{new }\ctext[RGB]{255,255,255}{tablet }\ctext[RGB]{255,255,255}{m

**Unique Identifier**: [urlsf_subset00]-[309279], OpenGPTText

In [20]:
SAMPLE1 = """Following the release of the iPad, the tablet market became a popular area for companies to explore. Despite many companies entering the market, no one has managed to outperform the iPad on sales.\nThere is a belief that Amazon and Google have caused this, due to the release of the Kindle Fire and Nexus 7 at $199, starting a \"race to the bottom.\" There is a concern that this will ensure there will be no profitability for anyone.\nHowever, this belief overlooks the fact that there is no proven market for tablets, only a proven market for iPads. Other than Apple, Samsung is the only company with notable tablet sales. Even though they released several tablets, in various shapes and sizes, they still could not compete with the iPad.\nHistorically, when prices are reduced so low that profits are not guaranteed, it is typically seen as a sign of a lack of market in general. This is what we are seeing in the tablet space. There is sufficient demand for iPads, but no one else has succeeded in creating a similar market.\nDespite this, Google and Amazon will likely both still make money from their tablets. However, they will not be made in the same way as with the iPad. For Amazon, they will make sales through content and other products, encouraged by Kindle Fire users. Google, on the other hand, may see success from mobile ads and through Android goodwill created by the Nexus 7.\nAsus, the maker of the Nexus 7, however, might not fare so well. Unless Google pays them sufficiently to make production of the tablet profitable, they may struggle to continue production. This is because Asus has no auxiliary sales from the Nexus 7, leaving them exposed to the race to the bottom. \nOverall, while there is demand for non-iPad tablets, there is no proof that consumers are willing to pay enough for them to make companies any money."""

evaluate(SAMPLE1, model, latex=True)

Predict as generated
With probability of 0.9999998807907104
LaTeX Renderer:


\ctext[RGB]{192,192,192}{Following }\ctext[RGB]{208,208,208}{the }\ctext[RGB]{225,225,225}{release }\ctext[RGB]{207,207,207}{of }\ctext[RGB]{207,207,207}{the }\ctext[RGB]{224,224,224}{iPad }\ctext[RGB]{193,193,193}{, }\ctext[RGB]{198,198,198}{the }\ctext[RGB]{228,228,228}{tablet }\ctext[RGB]{219,219,219}{market }\ctext[RGB]{194,194,194}{became }\ctext[RGB]{204,204,204}{ }\ctext[RGB]{219,219,219}{a }\ctext[RGB]{202,202,202}{popular }\ctext[RGB]{214,214,214}{area }\ctext[RGB]{208,208,208}{for }\ctext[RGB]{212,212,212}{companies }\ctext[RGB]{204,204,204}{to }\ctext[RGB]{192,192,192}{explore }\ctext[RGB]{195,195,195}{. }\ctext[RGB]{213,213,213}{ }\ctext[RGB]{189,189,189}{Despite }\ctext[RGB]{208,208,208}{many }\ctext[RGB]{197,197,197}{companies }\ctext[RGB]{196,196,196}{entering }\ctext[RGB]{210,210,210}{the }\ctext[RGB]{200,200,200}{market }\ctext[RGB]{212,212,212}{, }\ctext[RGB]{202,202,202}{no }\ctext[RGB]{201

In [24]:
TEST = """
What an astute inquiry! Our mentor proposed removing the initial and final sentences in the input data as a means of conducting the derbies, but we required additional time to test this approach methodically. Additionally, our team shares your apprehension about the precision of grammatically correct English and we will be presenting a test of our model's performance under both scenarios. Please note that this is merely a sample, but it may offer some valuable insight into our model's performance in the aforementioned settings.
"""
evaluate(TEST, model)

Predict as generated
With probability of 0.8621132969856262
